In [225]:
import xarray as xr
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling
import rasterio
import numpy as np

In [226]:
flood = xr.open_rasterio('DepthForecast.tif')
flood = flood.squeeze()

/var/folders/dx/yyl67b316vng4r9mgmc49gmw0000gn/T/ipykernel_99357/696510621.py:1: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  flood = xr.open_rasterio('DepthForecast.tif')


In [227]:
forecast = gpd.read_file('FloodExtentForecast.shp')

In [228]:
samples = gpd.read_file('subset_2shp.shp')
flood_labels = gpd.read_file('real_floodForecast.shp')
non_flood_labels = gpd.read_file('real_nonflood.shp')
sam_labels = gpd.sjoin(samples, labels, how='left', op='intersects')

/Users/sg/miniconda3/envs/ml/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3309: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [229]:
samples

,Shape_Leng,Shape_Area,no,geometry
0,2160.0,291600.0,1,"POLYGON ((761680.787 9618108.062, 761140.787 9..."
1,2160.0,291600.0,2,"POLYGON ((756820.787 9632688.062, 756820.787 9..."
2,2160.0,291600.0,3,"POLYGON ((763300.787 9617028.062, 762760.787 9..."
3,2160.0,291600.0,4,"POLYGON ((767080.787 9627288.062, 766540.787 9..."
4,2160.0,291600.0,5,"POLYGON ((764920.787 9628908.062, 764380.787 9..."
5,2160.0,291600.0,6,"POLYGON ((759520.787 9635928.062, 758980.787 9..."
6,2160.0,291600.0,7,"POLYGON ((760600.787 9632148.062, 760060.787 9..."
7,2160.0,291600.0,8,"POLYGON ((756820.787 9635388.062, 756280.787 9..."
8,2160.0,291600.0,9,"POLYGON ((760060.787 9615408.062, 759520.787 9..."
9,2160.0,291600.0,10,"POLYGON ((759520.787 9629448.062, 758980.787 9..."


In [230]:
sam_labels

,Shape_Leng,Shape_Area,no,geometry,index_right,Id,ID_sample,area
0,2160.0,291600.0,1,"POLYGON ((761680.787 9618108.062, 761140.787 9...",23.0,0.0,0.0,200583.736112
1,2160.0,291600.0,2,"POLYGON ((756820.787 9632688.062, 756820.787 9...",3.0,0.0,1.0,291599.999998
2,2160.0,291600.0,3,"POLYGON ((763300.787 9617028.062, 762760.787 9...",27.0,0.0,2.0,43349.786931
3,2160.0,291600.0,4,"POLYGON ((767080.787 9627288.062, 766540.787 9...",30.0,0.0,3.0,11736.453471
4,2160.0,291600.0,5,"POLYGON ((764920.787 9628908.062, 764380.787 9...",15.0,0.0,4.0,199287.355175
4,2160.0,291600.0,5,"POLYGON ((764920.787 9628908.062, 764380.787 9...",14.0,0.0,21.0,241300.961906
5,2160.0,291600.0,6,"POLYGON ((759520.787 9635928.062, 758980.787 9...",11.0,0.0,5.0,291600.000001
6,2160.0,291600.0,7,"POLYGON ((760600.787 9632148.062, 760060.787 9...",9.0,0.0,6.0,291600.000000
7,2160.0,291600.0,8,"POLYGON ((756820.787 9635388.062, 756280.787 9...",2.0,0.0,7.0,291600.000001
8,2160.0,291600.0,9,"POLYGON ((760060.787 9615408.062, 759520.787 9...",25.0,0.0,8.0,291600.000000


## Intesect evaluator/ model  Fmod - Feval

## What should it be a flood from evaluator and isnt from model (substraction)? Feval - Fmod

In [291]:
def flood_evaluator(flood_labels, forecast):
    flood_labels['area'] = flood_labels.area
    metrics_list = []
    for sample_id in flood_labels.ID_sample:
        print(sample_id)
        single_sample = flood_labels[flood_labels.ID_sample == sample_id]
        # Clean and validate the geometries
        forecast = forecast.buffer(0)
        single_sample_geom = single_sample.buffer(0)
        # Clip the forecast shapefile using the sample_1 shapefile
        clipped = gpd.clip(forecast, single_sample_geom)
        flooded_area_model = clipped.area.values.sum()
        #diff1 = flooded_area_model - single_sample['area'].values[0]
        #common_area = single_sample['area'].values[0] - diff1
        diff2 = single_sample['area'].values[0] - flooded_area_model
        metrics_list.append([sample_id, single_sample['area'].values[0], flooded_area_model, diff2])
    df1 = pd.DataFrame(metrics_list, columns=['ID_sample', 'Feval', 'Fmod_Feval', 'NFmod_Fe'])
    return(df1)

In [292]:
df1 = flood_evaluator(flood_labels, forecast)

23
26
7
1
25
27
11
24
9
6
32
5
20
29
21
4
13
14
30
12
10
17
28
0
31
8
33
2
19
15
3
16


In [293]:
df1

,ID_sample,Feval,Fmod_Feval,NFmod_Fe
0,23,291599.999998,291599.999998,0.000000
1,26,291600.000000,291600.000000,0.000000
2,7,291600.000001,291600.000001,0.000000
3,1,291599.999998,291599.999998,0.000000
4,25,291600.000001,291600.000001,0.000000
5,27,291599.999999,291599.999999,0.000000
6,11,291600.000000,291600.000000,0.000000
7,24,213274.028092,138383.875374,74890.152717
8,9,291600.000000,289133.770972,2466.229028
9,6,291600.000000,291600.000000,0.000000


In [263]:
df1.to_csv('forecast_report_flooded.csv')

## What is non flood according to  model and non flood according to the evaluator

In [239]:
non_flood_labels['Shape_Area'] = non_flood_labels.area

In [240]:
non_flood_labels

,Shape_Leng,Shape_Area,no,geometry
0,2160.0,9.101626e+04,1,"MULTIPOLYGON (((761140.787 9618108.062, 761140..."
1,2160.0,1.508743e-06,2,"POLYGON ((756820.787 9632688.062, 757360.787 9..."
2,2160.0,2.482502e+05,3,"MULTIPOLYGON (((762760.787 9617028.062, 762760..."
3,2160.0,2.798635e+05,4,"POLYGON ((766540.787 9627288.062, 766540.787 9..."
4,2160.0,9.231264e+04,5,"POLYGON ((764380.787 9628908.062, 764380.787 9..."
5,2160.0,1.508743e-06,6,"POLYGON ((758980.787 9635928.062, 758980.787 9..."
6,2160.0,1.508743e-06,7,"POLYGON ((760060.787 9632148.062, 760060.787 9..."
7,2160.0,5.029142e-07,8,"POLYGON ((756280.787 9635388.062, 756280.787 9..."
8,2160.0,5.029142e-07,9,"POLYGON ((759520.787 9615408.062, 759520.787 9..."
9,2160.0,5.029142e-07,10,"POLYGON ((758980.787 9629448.062, 758980.787 9..."


In [241]:
def non_flood_evaluator(non_flood_labels, forecast):
    non_flood_labels['Shape_Area'] = non_flood_labels.area
    metrics_list = []
    for sample_no in non_flood_labels.no:
        print(sample_no)
        single_sample = non_flood_labels[non_flood_labels.no == sample_no]
        if single_sample.Shape_Area.values[0]>0.001:
            # Clean and validate the geometries
            forecast = forecast.buffer(0)
            single_sample_geom = single_sample.buffer(0)
            # Clip the forecast shapefile using the sample_1 shapefile
            clipped = gpd.clip(forecast, single_sample_geom)
            flooded_area_model = clipped.area.values.sum()
            non_flooded_model = single_sample.Shape_Area.values[0] - flooded_area_model
            diff = single_sample.Shape_Area.values[0] - non_flooded_model
            metrics_list.append([sample_no, single_sample.Shape_Area.values[0], non_flooded_model, diff])
    df2 = pd.DataFrame(metrics_list, columns=['no', 'NFeval', 'NFeval_NFmod', 'Fmod_NFeval'])
    print(df2)
    return df2

In [296]:
non_flood_labels[non_flood_labels.Shape_Area<0.001]

,Shape_Leng,Shape_Area,no,geometry
1,2160.0,1.508743e-06,2,"POLYGON ((756820.787 9632688.062, 757360.787 9..."
5,2160.0,1.508743e-06,6,"POLYGON ((758980.787 9635928.062, 758980.787 9..."
6,2160.0,1.508743e-06,7,"POLYGON ((760060.787 9632148.062, 760060.787 9..."
7,2160.0,5.029142e-07,8,"POLYGON ((756280.787 9635388.062, 756280.787 9..."
8,2160.0,5.029142e-07,9,"POLYGON ((759520.787 9615408.062, 759520.787 9..."
9,2160.0,5.029142e-07,10,"POLYGON ((758980.787 9629448.062, 758980.787 9..."
19,2160.0,1.508743e-06,24,"POLYGON ((757360.787 9637008.062, 757360.787 9..."
21,2160.0,5.029142e-07,26,"POLYGON ((756280.787 9631608.062, 756280.787 9..."
22,2160.0,5.029142e-07,27,"POLYGON ((755200.787 9634848.062, 755200.787 9..."
23,2160.0,5.029142e-07,28,"POLYGON ((756280.787 9630528.062, 756280.787 9..."


In [205]:
df2 = non_flood_evaluator(non_flood_labels, forecast)

1
2
3
4
5
6
7
8
9
10
13
14
16
17
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


In [242]:
df2.to_csv('forecast_report_non_flooded.csv')

In [207]:
t100d2t = gpd.read_file('t100/t100d2t.shp')

In [208]:
report1 = flood_evaluator(flood_labels, t100d2t)
report2 = non_flood_evaluator(non_flood_labels, t100d2t)

23
26
7
1
25
27
11
24
9
6
32
5
20
29
21
4
13
14
30
12
10
17
28
0
31
8
33
2
19
15
3
16
1
2
3
4
5
6
7
8
9
10
13
14
16
17
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


In [220]:
report1.to_csv('t100/t100d2t_report_flooded.csv')
report2.to_csv('t100/t100d2t_report_non_flooded.csv')

In [222]:
t100d3t = gpd.read_file('t100/t100d3.shp')

In [223]:
report3 = flood_evaluator(flood_labels, t100d3t)
report4 = non_flood_evaluator(non_flood_labels, t100d3t)

23
26
7
1
25
27
11
24
9
6
32
5
20
29
21
4
13
14
30
12
10
17
28
0
31
8
33
2
19
15
3
16
1
2
3
4
5
6
7
8
9
10
13
14
16
17
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
    no         NFeval   NFeval_NFmod   Fmod_NFeval
0    1   91016.263888   73503.614158  17512.649730
1    3  248250.213069  235829.987695  12420.225374
2    4  279863.546529  273008.037513   6855.509016
3    5   92312.644825   90721.425826   1591.218999
4   13   54070.271844   51945.065723   2125.206120
5   14  123571.266099  121046.563773   2524.702326
6   16  253637.983313  248719.296171   4918.687142
7   17  118221.360491  113618.005748   4603.354743
8   19  291600.000000  290639.645035    960.354965
9   20   37265.744812   34315.972068   2949.772744
10  21   87167.707641   85393.977474   1773.730167
11  22   50299.038094   48632.054008   1666.984086
12  23  291600.000000  291600.000000      0.000000
13  25   78688.438704   62731.935329  15956.503376
14  29   21612.978300   17728.011201   3884.967100
15  31   88225.34

In [224]:
report3.to_csv('t100/t100d3_report_flooded.csv')
report4.to_csv('t100/t100d3_report_non_flooded.csv')

## P1-1b

In [305]:
subset = gpd.read_file('final/subset_2.shp')

In [315]:
subset

,Shape_Leng,Shape_Area,no,geometry
0,2160.0,291600.0,1,"POLYGON ((761680.787 9618108.062, 761140.787 9..."
1,2160.0,291600.0,2,"POLYGON ((756820.787 9632688.062, 756820.787 9..."
2,2160.0,291600.0,3,"POLYGON ((763300.787 9617028.062, 762760.787 9..."
3,2160.0,291600.0,4,"POLYGON ((767080.787 9627288.062, 766540.787 9..."
4,2160.0,291600.0,5,"POLYGON ((764920.787 9628908.062, 764380.787 9..."
5,2160.0,291600.0,6,"POLYGON ((759520.787 9635928.062, 758980.787 9..."
6,2160.0,291600.0,7,"POLYGON ((760600.787 9632148.062, 760060.787 9..."
7,2160.0,291600.0,8,"POLYGON ((756820.787 9635388.062, 756280.787 9..."
8,2160.0,291600.0,9,"POLYGON ((760060.787 9615408.062, 759520.787 9..."
9,2160.0,291600.0,10,"POLYGON ((759520.787 9629448.062, 758980.787 9..."


In [324]:
flood_labels

,Id,ID_sample,area,geometry
0,0,23,291599.999997,"POLYGON ((757900.786 9637548.062, 757900.786 9..."
1,0,26,291600.000000,"POLYGON ((755740.787 9634848.062, 755200.787 9..."
2,0,7,291600.000001,"POLYGON ((756820.787 9635388.062, 756280.787 9..."
3,0,1,291599.999998,"POLYGON ((757360.786 9633228.062, 757360.786 9..."
4,0,25,291600.000001,"POLYGON ((756820.787 9631608.062, 756280.787 9..."
5,0,27,291599.999999,"POLYGON ((756820.787 9630528.062, 756280.787 9..."
6,0,11,291600.000001,"POLYGON ((757900.786 9634308.062, 757360.786 9..."
7,0,24,212911.561297,"POLYGON ((758440.787 9631068.062, 758046.341 9..."
8,0,9,291600.000000,"POLYGON ((759520.787 9629448.062, 758980.787 9..."
9,0,6,291599.999999,"POLYGON ((760600.786 9632148.062, 760060.787 9..."


In [298]:
flood_labels = gpd.read_file('final/real_floodFORECAST_right.shp')
non_flood_labels = gpd.read_file('final/real_non_floodFORECAST_right.shp')
forecast = gpd.read_file('final/flood_extent/P1-1b/FloodExtentForecast.shp')

In [299]:
df11b_F = flood_evaluator(flood_labels, forecast)
df11b_NF = non_flood_evaluator(non_flood_labels, forecast)

23
26
7
1
25
27
11
24
9
6
32
5
20
29
21
4
13
14
30
12
10
17
28
0
31
8
33
2
19
15
3
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
16
17
18
19
20
21
22
23
24
25
26
27
28
29
31
32
33
34
    no         NFeval   NFeval_NFmod   Fmod_NFeval
0    1   91016.263888   64987.045400  26029.218488
1    3  248250.213068  234630.705418  13619.507651
2    4  279863.546529  267298.015423  12565.531106
3    5   92312.644825   73106.306191  19206.338634
4   13   54070.271843   30599.686132  23470.585711
5   14  123571.266100  111908.447345  11662.818755
6   16  253637.983313  249334.976786   4303.006527
7   17  118221.360491   92926.657027  25294.703464
8   19  291600.000000  290639.645035    960.354965
9   20   37265.744812    6509.522157  30756.222655
10  21   87167.707642   62105.584175  25062.123466
11  22   50299.038094   15233.742845  35065.295249
12  23  291600.000000  289622.287983   1977.712017
13  25   78688.438704   59232.173578  19456.265126
14  29   21612.978301   14173.528404   7439.449897
15  31   88

## P121b

In [422]:
flood_labels = gpd.read_file('final/real_floodD2_right.shp')
non_flood_labels = gpd.read_file('final/real_nonfloodD2_right.shp')
forecast = gpd.read_file('final/flood_extent/P1-2.1b/FloodExtentT100d2.shp')

In [423]:
df11b_F = flood_evaluator(flood_labels, forecast)
df11b_NF = non_flood_evaluator(non_flood_labels, forecast)

23
26
7
1
25
27
11
24
9
6
32
5
16
20
29
21
4
13
14
30
12
10
17
28
0
31
8
33
2
19
15
3
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
    no         NFeval   NFeval_NFmod   Fmod_NFeval
0    1  143468.433239   96771.586584  46696.846654
1    2   15451.190095     583.243245  14867.946850
2    3  262728.067585  246689.386739  16038.680846
3    4  286260.659546  278500.811799   7759.847747
4    5  237777.703895  204895.313637  32882.390259
5    6  133230.344521   83408.216229  49822.128292
6    7  169571.774069  124594.617724  44977.156346
7    8   63608.017463   39562.557893  24045.459571
8   10  173570.678603  140606.875214  32963.803389
9   11   30036.656516   17050.440731  12986.215786
10  12  228222.903197  203651.264681  24571.638516
11  13  147326.525800  122764.064708  24562.461093
12  14  226356.667395  221347.963945   5008.703450
13  15   19536.027026    3345.887588  16190.139438
14  16  260681.959232  255137.497371   5544.461860
15  1

In [426]:
df11b_NF

,no,NFeval,NFeval_NFmod,Fmod_NFeval
0,1,143468.433239,96771.586584,46696.846654
1,2,15451.190095,583.243245,14867.946850
2,3,262728.067585,246689.386739,16038.680846
3,4,286260.659546,278500.811799,7759.847747
4,5,237777.703895,204895.313637,32882.390259
5,6,133230.344521,83408.216229,49822.128292
6,7,169571.774069,124594.617724,44977.156346
7,8,63608.017463,39562.557893,24045.459571
8,10,173570.678603,140606.875214,32963.803389
9,11,30036.656516,17050.440731,12986.215786


## P122b

In [447]:
flood_labels = gpd.read_file('final/real_floodD3_right.shp')
non_flood_labels = gpd.read_file('final/real_nonfloodD3_right.shp')
forecast = gpd.read_file('final/flood_extent/P1-2.2b/FloodExtentT100d3.shp')

In [448]:
df11b_F = flood_evaluator(flood_labels, forecast)
df11b_NF = non_flood_evaluator(non_flood_labels, forecast)

23
26
7
1
25
27
11
24
9
6
32
5
20
29
21
4
13
14
30
12
10
17
28
0
31
8
33
2
19
15
3
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
    no         NFeval   NFeval_NFmod   Fmod_NFeval
0    1  160477.104198  106069.383625  54407.720573
1    2   84026.210354    4332.219938  79693.990416
2    3  262728.067585  246689.386740  16038.680845
3    4  286260.659546  276580.101870   9680.557676
4    5  254169.138228  212578.317283  41590.820946
5    6  133230.344521   80527.151332  52703.193190
6    7  186848.182673  137440.465290  49407.717382
7    8  106908.617894   73744.304991  33164.312902
8   10  182781.166229  149791.060798  32990.105431
9   12  249023.477139  215578.148632  33445.328507
10  13  147326.525800  114479.543004  32846.982796
11  14  208101.400933  203301.148528   4800.252406
12  15    7902.626818    2771.151843   5131.474974
13  16  260681.959232  255137.497371   5544.461860
14  17  291600.000000  248061.732038  43538.267962
15  18  

## Joins

In [449]:
joined = gpd.sjoin(subset, flood_labels, how='left', op='intersects')

/Users/sg/miniconda3/envs/ml/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3309: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [450]:
joined_ = joined.reset_index()

In [451]:
joined_

,index,Shape_Leng,Shape_Area,no,geometry,index_right,Id,ID_sample,area
0,0,2160.0,291600.0,1,"POLYGON ((761680.787 9618108.062, 761140.787 9...",23.0,0.0,0.0,131122.895803
1,1,2160.0,291600.0,2,"POLYGON ((756820.787 9632688.062, 756820.787 9...",3.0,0.0,1.0,207573.789647
2,2,2160.0,291600.0,3,"POLYGON ((763300.787 9617028.062, 762760.787 9...",27.0,0.0,2.0,28871.932415
3,3,2160.0,291600.0,4,"POLYGON ((767080.787 9627288.062, 766540.787 9...",30.0,0.0,3.0,5339.340454
4,4,2160.0,291600.0,5,"POLYGON ((764920.787 9628908.062, 764380.787 9...",15.0,0.0,4.0,37430.861772
5,5,2160.0,291600.0,6,"POLYGON ((759520.787 9635928.062, 758980.787 9...",11.0,0.0,5.0,158369.655479
6,6,2160.0,291600.0,7,"POLYGON ((760600.787 9632148.062, 760060.787 9...",9.0,0.0,6.0,104751.817328
7,7,2160.0,291600.0,8,"POLYGON ((756820.787 9635388.062, 756280.787 9...",2.0,0.0,7.0,184691.382107
8,8,2160.0,291600.0,9,"POLYGON ((760060.787 9615408.062, 759520.787 9...",25.0,0.0,8.0,291600.000000
9,9,2160.0,291600.0,10,"POLYGON ((759520.787 9629448.062, 758980.787 9...",8.0,0.0,9.0,108818.833771


In [452]:
joined_[joined_.duplicated(subset=['ID_sample'])]

,index,Shape_Leng,Shape_Area,no,geometry,index_right,Id,ID_sample,area
18,17,2160.0,291600.0,18,"POLYGON ((760600.787 9623508.062, 761140.787 9...",20.0,0.0,10.0,291600.000001
19,17,2160.0,291600.0,18,"POLYGON ((760600.787 9623508.062, 761140.787 9...",21.0,0.0,17.0,262862.058198
20,18,2160.0,291600.0,19,"POLYGON ((765460.787 9625128.062, 764920.787 9...",NaN,NaN,NaN,NaN
24,22,2160.0,291600.0,23,"POLYGON ((767620.787 9631608.062, 767080.787 9...",NaN,NaN,NaN,NaN
32,30,2160.0,291600.0,31,"POLYGON ((762220.787 9625668.062, 761680.787 9...",17.0,0.0,14.0,283697.373183


In [455]:
joined_[joined_.ID_sample == 14]

,index,Shape_Leng,Shape_Area,no,geometry,index_right,Id,ID_sample,area
15,14,2160.0,291600.0,15,"POLYGON ((761680.787 9625668.062, 761140.787 9...",17.0,0.0,14.0,283697.373183
32,30,2160.0,291600.0,31,"POLYGON ((762220.787 9625668.062, 761680.787 9...",17.0,0.0,14.0,283697.373183


In [456]:
joined_d = joined_.drop(index=18)

In [457]:
joined_dd = joined_d.drop(index=11)

In [458]:
joined_dd = joined_dd.drop(index=32)

In [459]:
joined_dd = joined_dd.reset_index()
joined_dd

,level_0,index,Shape_Leng,Shape_Area,no,geometry,index_right,Id,ID_sample,area
0,0,0,2160.0,291600.0,1,"POLYGON ((761680.787 9618108.062, 761140.787 9...",23.0,0.0,0.0,131122.895803
1,1,1,2160.0,291600.0,2,"POLYGON ((756820.787 9632688.062, 756820.787 9...",3.0,0.0,1.0,207573.789647
2,2,2,2160.0,291600.0,3,"POLYGON ((763300.787 9617028.062, 762760.787 9...",27.0,0.0,2.0,28871.932415
3,3,3,2160.0,291600.0,4,"POLYGON ((767080.787 9627288.062, 766540.787 9...",30.0,0.0,3.0,5339.340454
4,4,4,2160.0,291600.0,5,"POLYGON ((764920.787 9628908.062, 764380.787 9...",15.0,0.0,4.0,37430.861772
5,5,5,2160.0,291600.0,6,"POLYGON ((759520.787 9635928.062, 758980.787 9...",11.0,0.0,5.0,158369.655479
6,6,6,2160.0,291600.0,7,"POLYGON ((760600.787 9632148.062, 760060.787 9...",9.0,0.0,6.0,104751.817328
7,7,7,2160.0,291600.0,8,"POLYGON ((756820.787 9635388.062, 756280.787 9...",2.0,0.0,7.0,184691.382107
8,8,8,2160.0,291600.0,9,"POLYGON ((760060.787 9615408.062, 759520.787 9...",25.0,0.0,8.0,291600.000000
9,9,9,2160.0,291600.0,10,"POLYGON ((759520.787 9629448.062, 758980.787 9...",8.0,0.0,9.0,108818.833771


In [460]:
joined_dd[joined_dd.duplicated(subset=['ID_sample'])]

,level_0,index,Shape_Leng,Shape_Area,no,geometry,index_right,Id,ID_sample,area
18,20,18,2160.0,291600.0,19,"POLYGON ((765460.787 9625128.062, 764920.787 9...",NaN,NaN,NaN,NaN
22,24,22,2160.0,291600.0,23,"POLYGON ((767620.787 9631608.062, 767080.787 9...",NaN,NaN,NaN,NaN


no -> subset, ID_sample -> evaluation set

## Start from here

In [461]:
df11b_F

,ID_sample,Feval,Fmod_Feval,NFmod_Fe
0,23,208167.956343,201188.244402,6979.711941
1,26,218381.534939,195977.650747,22403.884192
2,7,184691.382107,120434.907711,64256.474396
3,1,207573.789647,207573.789647,0.000000
4,25,291600.000001,261942.041993,29657.958008
5,27,243818.824678,235193.808888,8625.015790
6,11,42576.522862,36792.106941,5784.415921
7,24,148964.810059,84792.683951,64172.126108
8,9,108818.833771,92835.393940,15983.439831
9,6,104751.817328,78052.399936,26699.417392


In [462]:
joined_2 = joined_dd.merge(df11b_F, left_on='ID_sample', right_on = 'ID_sample', how = 'left')
joined_2

,level_0,index,Shape_Leng,Shape_Area,no,geometry,index_right,Id,ID_sample,area,Feval,Fmod_Feval,NFmod_Fe
0,0,0,2160.0,291600.0,1,"POLYGON ((761680.787 9618108.062, 761140.787 9...",23.0,0.0,0.0,131122.895803,131122.895803,107275.662251,23847.233552
1,1,1,2160.0,291600.0,2,"POLYGON ((756820.787 9632688.062, 756820.787 9...",3.0,0.0,1.0,207573.789647,207573.789647,207573.789647,0.000000
2,2,2,2160.0,291600.0,3,"POLYGON ((763300.787 9617028.062, 762760.787 9...",27.0,0.0,2.0,28871.932415,28871.932415,5132.132214,23739.800201
3,3,3,2160.0,291600.0,4,"POLYGON ((767080.787 9627288.062, 766540.787 9...",30.0,0.0,3.0,5339.340454,5339.340454,3694.745567,1644.594887
4,4,4,2160.0,291600.0,5,"POLYGON ((764920.787 9628908.062, 764380.787 9...",15.0,0.0,4.0,37430.861772,37430.861772,15656.493779,21774.367992
5,5,5,2160.0,291600.0,6,"POLYGON ((759520.787 9635928.062, 758980.787 9...",11.0,0.0,5.0,158369.655479,158369.655479,149747.276642,8622.378837
6,6,6,2160.0,291600.0,7,"POLYGON ((760600.787 9632148.062, 760060.787 9...",9.0,0.0,6.0,104751.817328,104751.817328,78052.399936,26699.417392
7,7,7,2160.0,291600.0,8,"POLYGON ((756820.787 9635388.062, 756280.787 9...",2.0,0.0,7.0,184691.382107,184691.382107,120434.907711,64256.474396
8,8,8,2160.0,291600.0,9,"POLYGON ((760060.787 9615408.062, 759520.787 9...",25.0,0.0,8.0,291600.000000,291600.000000,291600.000000,0.000000
9,9,9,2160.0,291600.0,10,"POLYGON ((759520.787 9629448.062, 758980.787 9...",8.0,0.0,9.0,108818.833771,108818.833771,92835.393940,15983.439831


In [463]:
joined_3 = joined_2.merge(df11b_NF, on='no', how = 'left')

In [464]:
joined_3

,level_0,index,Shape_Leng,Shape_Area,no,geometry,index_right,Id,ID_sample,area,Feval,Fmod_Feval,NFmod_Fe,NFeval,NFeval_NFmod,Fmod_NFeval
0,0,0,2160.0,291600.0,1,"POLYGON ((761680.787 9618108.062, 761140.787 9...",23.0,0.0,0.0,131122.895803,131122.895803,107275.662251,23847.233552,160477.104198,106069.383625,54407.720573
1,1,1,2160.0,291600.0,2,"POLYGON ((756820.787 9632688.062, 756820.787 9...",3.0,0.0,1.0,207573.789647,207573.789647,207573.789647,0.000000,84026.210354,4332.219938,79693.990416
2,2,2,2160.0,291600.0,3,"POLYGON ((763300.787 9617028.062, 762760.787 9...",27.0,0.0,2.0,28871.932415,28871.932415,5132.132214,23739.800201,262728.067585,246689.386740,16038.680845
3,3,3,2160.0,291600.0,4,"POLYGON ((767080.787 9627288.062, 766540.787 9...",30.0,0.0,3.0,5339.340454,5339.340454,3694.745567,1644.594887,286260.659546,276580.101870,9680.557676
4,4,4,2160.0,291600.0,5,"POLYGON ((764920.787 9628908.062, 764380.787 9...",15.0,0.0,4.0,37430.861772,37430.861772,15656.493779,21774.367992,254169.138228,212578.317283,41590.820946
5,5,5,2160.0,291600.0,6,"POLYGON ((759520.787 9635928.062, 758980.787 9...",11.0,0.0,5.0,158369.655479,158369.655479,149747.276642,8622.378837,133230.344521,80527.151332,52703.193190
6,6,6,2160.0,291600.0,7,"POLYGON ((760600.787 9632148.062, 760060.787 9...",9.0,0.0,6.0,104751.817328,104751.817328,78052.399936,26699.417392,186848.182673,137440.465290,49407.717382
7,7,7,2160.0,291600.0,8,"POLYGON ((756820.787 9635388.062, 756280.787 9...",2.0,0.0,7.0,184691.382107,184691.382107,120434.907711,64256.474396,106908.617894,73744.304991,33164.312902
8,8,8,2160.0,291600.0,9,"POLYGON ((760060.787 9615408.062, 759520.787 9...",25.0,0.0,8.0,291600.000000,291600.000000,291600.000000,0.000000,NaN,NaN,NaN
9,9,9,2160.0,291600.0,10,"POLYGON ((759520.787 9629448.062, 758980.787 9...",8.0,0.0,9.0,108818.833771,108818.833771,92835.393940,15983.439831,182781.166229,149791.060798,32990.105431


In [465]:
joined_final = joined_3[['no', 'ID_sample', 'Feval', 'Fmod_Feval', 'NFmod_Fe', 'NFeval',
       'NFeval_NFmod', 'Fmod_NFeval']]

In [466]:
joined_final

,no,ID_sample,Feval,Fmod_Feval,NFmod_Fe,NFeval,NFeval_NFmod,Fmod_NFeval
0,1,0.0,131122.895803,107275.662251,23847.233552,160477.104198,106069.383625,54407.720573
1,2,1.0,207573.789647,207573.789647,0.000000,84026.210354,4332.219938,79693.990416
2,3,2.0,28871.932415,5132.132214,23739.800201,262728.067585,246689.386740,16038.680845
3,4,3.0,5339.340454,3694.745567,1644.594887,286260.659546,276580.101870,9680.557676
4,5,4.0,37430.861772,15656.493779,21774.367992,254169.138228,212578.317283,41590.820946
5,6,5.0,158369.655479,149747.276642,8622.378837,133230.344521,80527.151332,52703.193190
6,7,6.0,104751.817328,78052.399936,26699.417392,186848.182673,137440.465290,49407.717382
7,8,7.0,184691.382107,120434.907711,64256.474396,106908.617894,73744.304991,33164.312902
8,9,8.0,291600.000000,291600.000000,0.000000,NaN,NaN,NaN
9,10,9.0,108818.833771,92835.393940,15983.439831,182781.166229,149791.060798,32990.105431


NFeval = nan, polygon evaluator = polygon subset
ID_sample = nan, subset polygon = non flooded

In [467]:
joined_final.to_csv('final/t100d3.csv')